### This script does the preparation of Budinger_2020/Bharat

The original dataset is at 
`../../../../../../datasets/projects/20210309_Saliba_covidFibrosis_ignacio.ibarra_malte.luecken/bharat_et_al_full.h5ad`
It was used for Wendisch et al. Here Malte/Nacho are refining it to be used in the HLCA

In [318]:
import scanpy as sc

In [ ]:
ad = sc.read_h5ad('bharat_et_al_full.h5ad')

In [ ]:
ad.shape

In [ ]:
ad

In [ ]:
ad.obs_names = list(ad.obs['Unnamed: 0'])
# ad.obs

In [ ]:
# the numbers in ad.X are raw counts

In [ ]:
ad.raw is None

In [ ]:
# ad.to_df()['A1BG'].describe()

In [ ]:
## check mito content quickly
import matplotlib.pyplot as plt

In [ ]:
ad.obs['% of mito genes'].describe()

In [ ]:
ad.obs

### Remove all HEALTHY cells (these are already in the atlas and for that reason readding them is redundant.

In [ ]:
ad.obs['orig.ident'].value_counts().shape

In [ ]:
ad = ad[ad.obs['Sample Status'] != 'Control',:]
ad.obs['Sample Status'] = 'COVID-19

In [ ]:
plt.hist(ad.obs['% of mito genes'], bins=20)

In [ ]:
filter_mt = False
if filter_mt:
    mt_thr = 20
    print('# before', ad.shape)
    ad = ad[ad.obs['% of mito genes'] < mt_thr,:]
    print('# after', ad.shape)

In [ ]:
plt.hist(ad.obs['% of mito genes'], bins=20)

# Make object lightweight

In [ ]:
del ad.uns
del ad.obsm
del ad.varm
del ad.obsp

# Clean adata.var


In [ ]:
for c in ad.var.columns:
    del ad.var[c]

### I don't think this could divided into multiple samples. No clear batch, su assuming everything got sequenced together.

In [ ]:
ad.obs[['Sample Source', 'Sample Name', 'Sample Status', 'orig.ident']].value_counts()

In [ ]:
ad.obs['dataset'] = 'Bundinger2021'
ad.obs['study'] = 'Bundinger2021'
ad.obs['original_celltype_ann'] = ad.obs['Cell Type']
ad.obs['condition'] = ad.obs['Sample Status']
ad.obs['subject_ID'] = ad.obs['Sample Name']
ad.obs['sample'] = ad.obs['orig.ident']

ad.obs = ad.obs[ad.obs.columns[-6:]]

In [ ]:
# this is based on the paraphrasing of the manuscript info
age_by_subject_ID = {'PMB 1': 53, 'PMB 2': 57, 'Case 1': 27}
sex_by_subject_ID = {'PMB 1': 'M', 'PMB 2': 'F', 'Case 1': 'F'}
ethn_by_subject_ID = {'PMB 1': None, 'PMB 2': None, 'Case 1': 'Latina'}
bmi_by_subject_ID = {'PMB 1': None, 'PMB 2': None, 'Case 1': 25.2}

In [ ]:
ad.obs['age'] = ad.obs['subject_ID'].map(age_by_subject_ID)
ad.obs['sex'] = ad.obs['subject_ID'].map(sex_by_subject_ID)
ad.obs['ethnicity'] = ad.obs['subject_ID'].map(ethn_by_subject_ID)
ad.obs['bmi'] = ad.obs['subject_ID'].map(bmi_by_subject_ID)
ad.obs

In [ ]:
ad.obs['sample'].value_counts().shape

In [ ]:
# subset to 2000 HVGs that Lisa uses

In [ ]:
import pandas as pd
gene_set = pd.read_csv('genes_for_mapping.csv')
# gene_set = set(gene_set['gene_symbols'])

In [ ]:
len(set(ad.var.index).intersection(gene_set['gene_symbols']))

In [ ]:
import preprocessing as pp

In [ ]:
ad_sub = pp.subset_and_pad_adata(gene_set, ad) 

In [ ]:
ad.shape, ad_sub.shape

In [ ]:
ad.obs['subject_ID'].value_counts()

### Write a full and subsetted version of h5ad

In [ ]:
# Clean adata.var
full_path = '../../../data/HLCA_extended/extension_datasets/ready/full/bharat.h5ad'
ad.write(full_path, compression='lzf')


In [ ]:
subsetted_path = '../../../data/HLCA_extended/extension_datasets/ready/subsetted/bharat_sub.h5ad'
ad_sub.write(subsetted_path, compression='lzf')

In [ ]:
ad.obs

### Modify access to be checked

In [ ]:
!chmod 777 $full_path
!chmod 777 $subsetted_path